In [26]:
!pip install codecarbon

/opt/conda/lib/python3.10/pty.py:89: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid, fd = os.forkpty()
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [27]:
# Import required libraries
import pandas as pd
import numpy as np
import random
import torch
from datasets import Dataset
from sklearn.metrics import f1_score
from transformers import (
    AutoModelForSequenceClassification,
    AutoTokenizer,
    Trainer,
    TrainingArguments,
    pipeline,
)
import matplotlib.pyplot as plt

In [28]:
# Load the dataset
splits = {'train': 'train.csv', 'test': 'test.csv'}
train_data = pd.read_csv("hf://datasets/holistic-ai/EMGSD/" + splits["train"])
test_data = pd.read_csv("hf://datasets/holistic-ai/EMGSD/" + splits["test"])

In [29]:
# Use a subset of the data for faster training
sample_ratio = 0.001
train_data = train_data.sample(frac=sample_ratio, random_state=42)
test_data = test_data.sample(frac=sample_ratio, random_state=42)

# Prepare train and test sets by using both training and testing data
X_train, y_train = train_data["text"].values.tolist(), train_data["category"].values.tolist()
X_test, y_test = test_data["text"].values.tolist(), test_data["category"].values.tolist()

In [30]:
# Function to compute Macro F1 score
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    f1 = f1_score(labels, predictions, average='macro')  # Use macro F1
    return {"f1": f1}

In [31]:
# Convert to Hugging Face dataset format
train_dataset = Dataset.from_dict({"text": X_train, "label": y_train})
test_dataset = Dataset.from_dict({"text": X_test, "label": y_test})

# Map labels to IDs
label2id = {
    'stereotype': 0,
    'unrelated': 1,
    'neutral': 2,
}

id2label = {v: k for k, v in label2id.items()}

def map_labels(example):
    example['label'] = label2id[example['label']]
    return example

# Apply the mapping to your dataset
train_dataset = train_dataset.map(map_labels)
test_dataset = test_dataset.map(map_labels)

# Random Model Prediction
random.seed(42)
random_predictions = [random.choice(y_test) for _ in range(len(y_test))]

# Evaluate the model
f1 = f1_score(y_test, random_predictions, average='macro')
print(f"F1 Score: {f1}")

Map:   0%|          | 0/46 [00:00<?, ? examples/s]

Map:   0%|          | 0/11 [00:00<?, ? examples/s]

F1 Score: 0.35714285714285715


In [32]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from codecarbon import EmissionsTracker

# TF-IDF Vectorizer
X_train = train_dataset['text']
y_train = train_dataset['label']  
X_test = test_dataset['text']
y_test = test_dataset['label']

vectorizer = TfidfVectorizer()
X_train_tfidf = vectorizer.fit_transform(X_train)
X_test_tfidf = vectorizer.transform(X_test)

# Logistic Regression Model
model = LogisticRegression()

# Tracking emissions with CodeCarbon
tracker = EmissionsTracker()
#tracker.start()

# Fit the model
model.fit(X_train_tfidf, y_train)

# Evaluate the model
predictions = model.predict(X_test_tfidf)
f1 = f1_score(y_test, predictions, average='macro')

#emissions = tracker.stop()
print(f"F1 Score: {f1}")
#print(f"Training carbon emissions: {emissions} kg")

[codecarbon ERROR @ 13:03:57] Error: Another instance of codecarbon is already running. Turn off the other instance to be able to run this one. Exiting.


F1 Score: 0.17777777777777778


In [33]:
import torch
from transformers import AutoModelForSequenceClassification, AutoTokenizer, Trainer, TrainingArguments
import matplotlib.pyplot as plt


# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained("albert-base-v2")

# Tokenization function
def tokenize_function(example):
    return tokenizer(example['text'], padding='max_length', truncation=True)
    
# Apply the tokenizer to the dataset
tokenized_train_dataset = train_dataset.map(tokenize_function, batched=True)
tokenized_test_dataset = test_dataset.map(tokenize_function, batched=True)

/opt/conda/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1617: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be deprecated in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


Map:   0%|          | 0/46 [00:00<?, ? examples/s]

Map:   0%|          | 0/11 [00:00<?, ? examples/s]

In [34]:
# Load pre-trained ALBERT model with classification head
#model = AutoModelForSequenceClassification.from_pretrained(
#    "albert-base-v2", 
#    num_labels=3, 
#    label2id=label2id,
#    id2label=id2label
#)

model = AutoModelForSequenceClassification.from_pretrained(
    "roberta-base",
    num_labels=3,
    label2id=label2id,
    id2label=id2label
)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu") # "mps" For macOS (Apple Silicon)
model.to(device)

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


RobertaForSequenceClassification(
  (roberta): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(50265, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0-11): 12 x RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
         

In [35]:
# Fine-tuning the model and save the best model
training_args = TrainingArguments(
    output_dir='./results',
    eval_strategy="epoch",
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    logging_dir='./logs',
    num_train_epochs=3,
    logging_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    save_total_limit=1,
    metric_for_best_model="f1",
    greater_is_better=True
)

trainer = Trainer(
    model=model,
    tokenizer=tokenizer,
    args=training_args,
    train_dataset=tokenized_train_dataset,
    eval_dataset=tokenized_test_dataset,
    compute_metrics=compute_metrics,  # Use macro F1 computation
)

[codecarbon ERROR @ 13:03:58] Error: Another instance of codecarbon is already running. Turn off the other instance to be able to run this one. Exiting.


In [36]:
# Tracking emissions with CodeCarbon
tracker = EmissionsTracker()
#tracker.start()

#trainer.train()

#emissions = tracker.stop()

print()
#print(f"Training carbon emissions: {emissions} kg")

[codecarbon ERROR @ 13:03:58] Error: Another instance of codecarbon is already running. Turn off the other instance to be able to run this one. Exiting.


In [37]:
!pip install nlpaug

/opt/conda/lib/python3.10/pty.py:89: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid, fd = os.forkpty()
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.5/410.5 kB 7.9 MB/s eta 0:00:00ta 0:00:01


In [124]:
import torch
from transformers import (
    AutoTokenizer,
    AutoConfig,
    Trainer,
    TrainingArguments,
    AdamW,
    get_linear_schedule_with_warmup
)
import numpy as np
from sklearn.metrics import accuracy_score, f1_score
from sklearn.utils.class_weight import compute_class_weight
from torch.nn import CrossEntropyLoss
from datasets import Dataset, concatenate_datasets
import pandas as pd
import re
from collections import Counter
from imblearn.over_sampling import RandomOverSampler
import nlpaug.augmenter.word as naw
import json

import os
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"

model_name = "distilroberta-base"
tokenizer = AutoTokenizer.from_pretrained(model_name)

import gc
from transformers import TrainerCallback

class GpuMemoryCallback(TrainerCallback):
    def on_epoch_end(self, args, state, control, **kwargs):
        allocated_memory = torch.cuda.memory_allocated() / 1024**2
        cached_memory = torch.cuda.memory_reserved() / 1024**2
        print(f"Epoch {state.epoch}:")
        print(f"Allocated GPU Memory: {allocated_memory:.2f} MB")
        print(f"Cached GPU Memory: {cached_memory:.2f} MB")
        print(torch.cuda.memory_summary())

class MemoryCleanupCallback(TrainerCallback):
    def on_epoch_end(self, args, state, control, **kwargs):
        print("Cleared cache")
        gc.collect()
        torch.cuda.empty_cache()
        torch.cuda.empty_cache()
        torch.cuda.empty_cache()
        torch.cuda.empty_cache()
        torch.cuda.empty_cache()


def cleanup_memory():
    gc.collect()
    torch.cuda.empty_cache()
    
# **1. Preprocess Text**
def preprocess_text(examples):
    text = examples['text']
    # Convert to lowercase
    text = text.lower()
    # Remove URLs
    text = re.sub(r'http\S+|www.\S+', '', text)
    # Remove special characters
    text = re.sub(r'[^a-zA-Z0-9\s]', '', text)
    examples['text'] = text
    return examples

# Apply preprocessing
train_dataset = train_dataset.map(preprocess_text)
test_dataset = test_dataset.map(preprocess_text)

# **2. Convert to Pandas DataFrame for Oversampling**
train_df = train_dataset.to_pandas()

# Separate features and labels
X = train_df[['text']]
y = train_df['label']

# **3. Over-Sample Minority Classes**
ros = RandomOverSampler(random_state=42)
X_resampled, y_resampled = ros.fit_resample(X, y)

# Convert back to Hugging Face Dataset
resampled_df = pd.concat([X_resampled, y_resampled], axis=1)
resampled_dataset = Dataset.from_pandas(resampled_df)

# **4. Data Augmentation**
def augment_text(examples):
    aug = naw.SynonymAug()
    augmented_texts = [" ".join(text) if isinstance(text, list) else text for text in examples['text']]
    return {"text": augmented_texts, "label": examples['label']}

# Apply augmentation
augmented_dataset = resampled_dataset.map(augment_text, batched=True)

# Combine with resampled dataset
combined_dataset = concatenate_datasets([resampled_dataset, augmented_dataset])

# **5. Tokenization**
def tokenize_function(examples):
    return tokenizer(
        examples['text'],
        padding='max_length',
        truncation=True,
        max_length=100  # Adjust to a lower value if possible
    )

# Ensure that tokenized dataset includes 'input_ids' and other keys required for training
tokenized_train_dataset = combined_dataset.map(tokenize_function, batched=True)
tokenized_test_dataset = test_dataset.map(tokenize_function, batched=True)

# Ensure the labels are correctly aligned in the dataset
tokenized_train_dataset = tokenized_train_dataset.rename_column("label", "labels")
tokenized_test_dataset = tokenized_test_dataset.rename_column("label", "labels")

# Set format to PyTorch tensors for compatibility with Trainer
tokenized_train_dataset.set_format(type="torch", columns=["input_ids", "attention_mask", "labels"])
tokenized_test_dataset.set_format(type="torch", columns=["input_ids", "attention_mask", "labels"])

# **6. Update Labels and Compute Class Weights**
train_labels = np.array(tokenized_train_dataset['labels'])

# Check class distribution
label_counts = Counter(train_labels)
print("Class distribution after oversampling and augmentation:", label_counts)

# Compute class weights
unique_classes = np.unique(train_labels).astype(int)
class_weights = compute_class_weight(
    class_weight='balanced',
    classes=unique_classes,
    y=train_labels
)
class_weights = class_weights / class_weights.sum()
class_weights = torch.tensor(class_weights, dtype=torch.float)

# **7. Determine Device**
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# **8. Load Config and Customize Model**
label2id = {int(i): f'class_{i}' for i in unique_classes}
id2label = {v: k for k, v in label2id.items()}

config = AutoConfig.from_pretrained(model_name, num_labels=len(unique_classes), label2id=label2id, id2label=id2label)

from transformers import RobertaForSequenceClassification
from transformers.modeling_outputs import SequenceClassifierOutput

class CustomRoberta(RobertaForSequenceClassification):
    def __init__(self, config):
        super().__init__(config)
        self.classifier = torch.nn.Linear(config.hidden_size, config.num_labels)

    def forward(self, input_ids=None, attention_mask=None, labels=None):
        outputs = self.roberta(
            input_ids=input_ids,
            attention_mask=attention_mask
        )
        sequence_output = outputs[0]  # [batch_size, seq_length, hidden_size]
        pooled_output = sequence_output[:, 0, :]  # [batch_size, hidden_size]
        logits = self.classifier(pooled_output)  # [batch_size, num_labels]

        return SequenceClassifierOutput(
            logits=logits,
            hidden_states=outputs.hidden_states,
            attentions=outputs.attentions,
        )

model = CustomRoberta.from_pretrained(model_name, config=config)
model = torch.nn.DataParallel(model)
model.to(device)
class_weights = class_weights.to(device)

# **9. Define Training Arguments**
training_args = TrainingArguments(
    output_dir='./results',
    evaluation_strategy="epoch",
    per_device_train_batch_size=1,
    per_device_eval_batch_size=1,
    gradient_accumulation_steps=4,
    num_train_epochs=5,
    learning_rate=1e-5,
    weight_decay=0.01,
    warmup_steps=100,
    logging_dir='./logs',
    logging_strategy="steps",
    logging_steps=50,
    save_strategy="epoch",
    load_best_model_at_end=True,
    save_total_limit=1,
    metric_for_best_model="f1",
    greater_is_better=True,
    fp16=False,  # Disable mixed precision to avoid errors with gradient scaling
    remove_unused_columns=False  # Ensure that all columns are preserved during training
)

# **10. Initialize Optimizer and Scheduler**
optimizer = AdamW(model.parameters(), lr=training_args.learning_rate, weight_decay=training_args.weight_decay)

total_steps = (
    len(tokenized_train_dataset) // training_args.per_device_train_batch_size
) * training_args.num_train_epochs

scheduler = get_linear_schedule_with_warmup(
    optimizer,
    num_warmup_steps=training_args.warmup_steps,
    num_training_steps=total_steps
)

# **11. Define Custom Trainer Class**
class CustomTrainer(Trainer):
    def compute_loss(self, model, inputs, return_outputs=False):
        labels = inputs.pop("labels").to(device)
        inputs = {k: v.to(device) for k, v in inputs.items()}
        outputs = model(**inputs)
        logits = outputs.logits

        # Compute custom loss with class weights on the correct device
        loss_fct = CrossEntropyLoss(weight=class_weights.to(device))
        loss = loss_fct(logits, labels)
        allocated_memory = torch.cuda.memory_allocated() / 1024**2
        reserved_memory = torch.cuda.memory_reserved() / 1024**2
        print(f"Allocated Memory: {allocated_memory:.2f} MB")
        print(f"Reserved Memory: {reserved_memory:.2f} MB")
        cleanup_memory()
        
        if return_outputs:
            return (loss, outputs)
        else:
            return loss

    def training_step(self, model, inputs, num_items_in_batch=None):
        model.train()
        inputs = self._prepare_inputs(inputs)
        loss = self.compute_loss(model, inputs)
        allocated_memory = torch.cuda.memory_allocated() / 1024**2
        reserved_memory = torch.cuda.memory_reserved() / 1024**2
        print(f"Allocated Memory: {allocated_memory:.2f} MB")
        print(f"Reserved Memory: {reserved_memory:.2f} MB")
        cleanup_memory()
        
        return loss

# **12. Define Compute Metrics Function**
def compute_metrics(p):
    preds = np.argmax(p.predictions, axis=1)
    return {
        'accuracy': accuracy_score(p.label_ids, preds),
        'f1': f1_score(p.label_ids, preds, average='macro')
    }

# **13. Initialize the Custom Trainer**
trainer = CustomTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train_dataset,
    eval_dataset=tokenized_test_dataset,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
    optimizers=(optimizer, scheduler),
    callbacks=[GpuMemoryCallback(), MemoryCleanupCallback()]
)

# Start training
trainer.train()


/opt/conda/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1617: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be deprecated in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


Map:   0%|          | 0/46 [00:00<?, ? examples/s]

Map:   0%|          | 0/11 [00:00<?, ? examples/s]

Map:   0%|          | 0/54 [00:00<?, ? examples/s]

[nltk_data] Downloading package wordnet to /usr/share/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /usr/share/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


...

Map:   0%|          | 0/11 [00:00<?, ? examples/s]

Class distribution after oversampling and augmentation: Counter({2: 36, 1: 36, 0: 36})


Some weights of CustomRoberta were not initialized from the model checkpoint at distilroberta-base and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/opt/conda/lib/python3.10/site-packages/transformers/training_args.py:1545: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/transformers/optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/accelerate/accelerator.py:494: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...

Allocated Memory: 2128.75 MB
Reserved Memory: 2296.00 MB
Allocated Memory: 2128.75 MB
Reserved Memory: 2276.00 MB
Allocated Memory: 2249.75 MB
Reserved Memory: 2296.00 MB
Allocated Memory: 2249.75 MB
Reserved Memory: 2296.00 MB
Allocated Memory: 2361.13 MB
Reserved Memory: 2370.00 MB
Allocated Memory: 2361.13 MB
Reserved Memory: 2368.00 MB
Allocated Memory: 2464.50 MB
Reserved Memory: 2482.00 MB
Allocated Memory: 2464.50 MB
Reserved Memory: 2480.00 MB


AssertionError: Attempted unscale_ but _scale is None.  This may indicate your script did not use scaler.scale(loss or outputs) earlier in the iteration.

In [115]:
torch.cuda.empty_cache()

In [122]:
# Start training
trainer.train()

[codecarbon WARNING @ 13:41:14] Another instance of codecarbon is already running. Exiting.
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/parallel_apply.py:79: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.device(device), torch.cuda.stream(stream), autocast(enabled=autocast_enabled):


Allocated Memory: 1714.74 MB
Reserved Memory: 1798.00 MB
Allocated Memory: 1714.74 MB
Reserved Memory: 1798.00 MB
Allocated Memory: 1835.74 MB
Reserved Memory: 1876.00 MB
Allocated Memory: 1835.74 MB
Reserved Memory: 1874.00 MB
Allocated Memory: 1946.24 MB
Reserved Memory: 1954.00 MB
Allocated Memory: 1946.24 MB
Reserved Memory: 1954.00 MB
Allocated Memory: 2050.49 MB
Reserved Memory: 2066.00 MB
Allocated Memory: 2050.49 MB
Reserved Memory: 2064.00 MB


AssertionError: Attempted unscale_ but _scale is None.  This may indicate your script did not use scaler.scale(loss or outputs) earlier in the iteration.

In [ ]:
from sklearn.metrics import f1_score

# Making predictions on the test set
preds = trainer.predict(tokenized_test_dataset).predictions.argmax(-1)
f1 = f1_score(tokenized_test_dataset['label'], preds, average='macro')
print(f"Macro F1 Score: {f1:.3f}")